In [1]:
library(prospectr)
library(reticulate)
library(dotenv)
library(jsonlite)

prospectr version 0.2.1 -- 'seville'

check the github repository at: http://github.com/l-ramirez-lopez/prospectr



In [2]:
# Split working directory path
pwd = getwd()
split_path = function(x) if (dirname(x)==x) x else c(basename(x),split_path(dirname(x)))
path_list = split_path(pwd)

# Change working directory
if (path_list[1] == "split"){
    new_pwd = ""
    for (i in 3:length(path_list)-1) {
        new_pwd = paste(path_list[i], new_pwd, sep=path_list[length(path_list)])  
    }
    new_pwd = paste(path_list[length(path_list)], new_pwd, sep="")
    setwd(new_pwd)
}

In [3]:
# Load ML Adapter data
load_dot_env(file = ".env")
file_path = Sys.getenv("ML_ADAPTER_DATA")
data = fromJSON(txt=file_path)

In [4]:
# Determine X by reading the .csv file
split_file = data$DATASET
dataset = read.csv(split_file, check.names=FALSE)
#print(dim(dataset))

In [5]:
# Filter dataset to contain only numeric columns
dataset_numeric = dataset[sapply(dataset, is.numeric)]
#print(dim(dataset_numeric))

In [6]:
# Determine N and k

split_methods = Sys.getenv("SPLIT")
split_methods = fromJSON(txt=split_methods)
N = split_methods$kennard_stone$N
k = split_methods$kennard_stone$k

In [7]:
# Take a sample of the dataset
if (nrow(dataset_numeric) > N){
    set.seed(10000)
    X = dataset_numeric[sample(nrow(dataset_numeric), N),]
    set.seed(10000)
    X_full = dataset[sample(nrow(dataset), N),]
} else{
    X = dataset_numeric
}  
#print(dim(X))

In [8]:
# Determine k proportionately if N is greater than the rows of X.
if (N > nrow(X)){
    fraction = k / N
    k = ceiling(nrow(X) * fraction)
    #print(fraction)
    #print(k)
}

In [9]:
# Run kennard stone algorithm
start_time <- Sys.time()
selection = kenStone(X, k=k, metric = "euclid")
end_time = Sys.time()
#print(end_time - start_time)

In [10]:
# Get training and testing file paths
pwd = getwd()
split_path = function(x) if (dirname(x)==x) x else c(basename(x),split_path(dirname(x)))
path_list = split_path(pwd)
training_path = paste(pwd, "data", "train.csv", sep=path_list[length(path_list)])
testing_path = paste(pwd, "data", "test.csv", sep=path_list[length(path_list)])

# Write training and testing sets to a csv file
write.csv(x=X_full[selection$model,], file=training_path, row.names=FALSE)
write.csv(x=X_full[selection$test,], file=testing_path, row.names=FALSE)